In [1]:
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.system("rm -r logs")
import tensorflow as tf
# get_ipython().run_line_magic('matplotlib', 'inline')
import matplotlib.pyplot as plt 
from PIL import Image
import multiprocessing

In [2]:
trainPath = '/home/winsoul/disk/fMRI/data/separatedData/mixedData/stimulationExceptStatic/tfrecords/train.tfrecords'
testPath = '/home/winsoul/disk/fMRI/data/separatedData/mixedData/stimulationExceptStatic/tfrecords/test.tfrecords'
valPath = '/home/winsoul/disk/fMRI/data/separatedData/mixedData/stimulationExceptStatic/tfrecords/val.tfrecords'
model_path = '/home/winsoul/disk/fMRI/HEATMAP/ComputerDesign/model/'

In [3]:
def read_tfrecord(TFRecordPath):
    with tf.Session() as sess:
        feature = {
            'image': tf.FixedLenFeature([], tf.string),
            'label': tf.FixedLenFeature([], tf.int64),
            'person': tf.FixedLenFeature([], tf.int64),
        }
#         filename_queue = tf.train.string_input_producer([TFRecordPath], num_epochs = 1)
        filename_queue = tf.train.string_input_producer([TFRecordPath])
        reader = tf.TFRecordReader()
        _, serialized_example = reader.read(filename_queue)
        features = tf.parse_single_example(serialized_example, features = feature)
        image = tf.decode_raw(features['image'], np.float64)
        image = tf.cast(image, tf.float32)
        image = tf.reshape(image, [31, 128, 128, 1])
        label = tf.cast(features['label'], tf.int32)
        return image, label - 1

In [4]:
# s 步长，channels_in 输入通道，channels_out 输出通道
def conv3d_layer(X, k, s, channels_in, channels_out, is_training, name = 'CONV'):
    with tf.name_scope(name):
        W = tf.Variable(tf.truncated_normal([k, k, k, channels_in, channels_out], stddev = 0.1));
        b = tf.Variable(tf.constant(0.1, shape = [channels_out]))
        conv = tf.nn.conv3d(X, W, strides = [1, s, s, s, 1], padding = 'SAME') + b
#         bn = tf.layers.batch_normalization(conv, training = is_training)
        result = tf.nn.relu(conv)
        tf.summary.histogram('weights', W)
        tf.summary.histogram('biases', b)
        tf.summary.histogram('activations', result)
        return result
    
def pool3d_layer(X, k, s, strr = 'SAME', pool_type = 'MAX', name = 'pool', down_stride = 1):
    if pool_type == 'MAX':
        result = tf.nn.max_pool3d(X,
                              ksize = [1, down_stride, k, k, 1],
                              strides = [1, down_stride, s, k, 1],
                              padding = strr,
                              name = name)
    else:
        result = tf.nn.avg_pool3d(X,
                              ksize = [1, down_stride, k, k, 1],
                              strides = [1, down_stride, s, k, 1],
                              padding = strr,
                              name = name)
    return result

def fc_layer(X, neurons_in, neurons_out, last = False, name = 'FC'):
    with tf.name_scope(name):
        W = tf.Variable(tf.truncated_normal([neurons_in, neurons_out], stddev = 0.1))
        b = tf.Variable(tf.constant(0.1, shape = [neurons_out]))
        tf.summary.histogram('weights', W)
        tf.summary.histogram('biases', b)
        if last == False:
            result = tf.nn.relu(tf.matmul(X, W) + b)
        else:
            result = tf.matmul(X, W) + b
        tf.summary.histogram('activations', result)
        return result

In [5]:
def Network(BatchSize, learning_rate):
    tf.reset_default_graph()
    with tf.Session() as sess:
        is_training = tf.placeholder(dtype = tf.bool, shape=())
        keep_prob = tf.placeholder('float32', name = 'keep_prob')
        
        judge = tf.Print(is_training, ['is_training:', is_training])
        
        image_train, label_train = read_tfrecord(trainPath) 
        image_val, label_val = read_tfrecord(valPath) 

        image_train_Batch, label_train_Batch = tf.train.shuffle_batch([image_train, label_train], 
                                                     batch_size = BatchSize, 
                                                     capacity = BatchSize*3 + 200,
                                                     min_after_dequeue = BatchSize)
        image_val_Batch, label_val_Batch = tf.train.shuffle_batch([image_val, label_val], 
                                                     batch_size = BatchSize, 
                                                     capacity = BatchSize*3 + 200,
                                                     min_after_dequeue = BatchSize)
        
        image_Batch = tf.cond(is_training, lambda: image_train_Batch, lambda: image_val_Batch)
        label_Batch = tf.cond(is_training, lambda: label_train_Batch, lambda: label_val_Batch)
        
        label_Batch = tf.one_hot(label_Batch, depth = 3)
        


        X = tf.identity(image_Batch)
        y = tf.identity(label_Batch)
    
        conv1_1 = conv3d_layer(X, 3, 1, 1, 16, is_training, "conv1_1")
        conv1_2 = conv3d_layer(conv1_1, 3, 1, 16, 16, is_training, "conv1_2")
#         conv1_3 = conv3d_layer(conv1_2, 3, 1, 16, 16, is_training, "conv1_3")
        pool1 = pool3d_layer(conv1_2, 2, 2, "SAME", "MAX", 'pool1')

        conv2_1 = conv3d_layer(pool1, 3, 1, 16, 32, is_training, 'conv2_1')
#         conv2_2 = conv3d_layer(conv2_1, 3, 1, 32, 32, is_training, 'conv2_2')
        pool2 = pool3d_layer(conv2_1, 2, 2, "SAME", "MAX", 'pool2')
        
        conv3 = conv3d_layer(pool2, 3, 1, 32, 16, is_training, 'conv3')
        pool3 = pool3d_layer(conv3, 2, 2, "SAME", "MAX", 'pool3', down_stride = 2)
        print(pool3.shape)
        
        drop1 = tf.nn.dropout(pool3, keep_prob)
        fc1 = fc_layer(tf.reshape(drop1, [-1, 16 * 16 * 16 * 16]), 16 * 16 * 16 * 16, 1024, name = 'fc1')
        
        drop2 = tf.nn.dropout(fc1, keep_prob)
        fc2 = fc_layer(drop2, 1024, 128, name = 'fc2')
        
        drop3 = tf.nn.dropout(fc2, keep_prob)
        y_result = fc_layer(drop3, 128, 3, True, name = 'fc3')
        
        
#         conv4 = conv3d_layer(pool3, 3, 1, 38, 3, 'conv4')
#         pool4 = pool3d_layer(conv4, 11, 11, "SAME", "MEAN", 'pool4')
#         print(pool4.shape)
        
#         y_result = tf.reshape(pool4, [-1, 3])
#         print(y_result.shape)
        
        with tf.name_scope('summaries'):
            update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
            with tf.control_dependencies(update_ops): 
    #             cross_entropy = -tf.reduce_mean(y * tf.log(tf.clip_by_value(y_result, 1e-10,1.0)))
                cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = y_result, labels = y))
                train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)
                #train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)
                corrent_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_result, 1))
                accuracy = tf.reduce_mean(tf.cast(corrent_prediction, 'float', name = 'accuracy'))
                tf.summary.scalar("loss", cross_entropy)
                tf.summary.scalar("accuracy", accuracy)
            
        init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
        sess.run(init_op)
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord = coord)
        
        merge_summary = tf.summary.merge_all()
        summary__train_writer = tf.summary.FileWriter("./logs/train" + '_rate:' + str(learning_rate) + '_' + str(BatchSize), sess.graph)
        summary_val_writer = tf.summary.FileWriter("./logs/test" + '_rate:' + str(learning_rate) + '_' + str(BatchSize))
        
        saver = tf.train.Saver()
        
        can_save_model = False
        try:
            batch_index = 1
            while not coord.should_stop():
                sess.run([train_step], feed_dict = {keep_prob: 0.5, is_training: True})
                if batch_index % 10 == 0:
                    summary_train, _, acc_train, loss_train = sess.run([merge_summary, train_step, accuracy, cross_entropy], feed_dict = {keep_prob: 0.5, is_training: True})   
                    summary__train_writer.add_summary(summary_train, batch_index) 
                    print(str(batch_index) + ' train:' + '  ' + str(acc_train) + ' ' + str(loss_train))
                    summary_val, acc_val, loss_val = sess.run([merge_summary, accuracy, cross_entropy], feed_dict = {keep_prob: 1.0, is_training: False}) 
                    summary_val_writer.add_summary(summary_val, batch_index) 
                    print(str(batch_index) + '  val: ' + '  ' + str(acc_val) + ' ' + str(loss_val))

                    if acc_val > 0.8 :
                        can_save_model = True
                    
#                 if can_save_model == True and batch_index % 100 == 0:
#                     save_path = saver.save(sess, model_path + 'Model__Step_{:04d}_{:d}_'.format(batch_index, BatchSize) + str(learning_rate))
                
#                 if can_save_model == False and batch_index % 1000 == 0:
#                     break
                    
#                 if can_save_model == True and batch_index % 1500 == 0:
#                     break
#                 if batch_index % 1500 == 0:
#                     break
                if batch_index % 1000 == 0:
                    save_path = saver.save(sess, model_path + 'Model__Step_{:04d}_{:d}_'.format(batch_index, BatchSize) + str(learning_rate))
                batch_index += 1;
                    
        except tf.errors.OutOfRangeError:
            print("OutofRangeError!")
        finally:
            print("Finish")
    
        coord.request_stop()
        coord.join(threads)
        sess.close()


In [ ]:
#1, 12, 48, 24, 512, 0.003
def main():
    Network(8, 0.0003)
#     rate = 0.01
#     batch = 8
#     while batch >= 1:
#         rate = 0.01
#         while rate > 0.00001:
#             print("---------------------------------------------------------------------")
#             print(batch, rate)
#             try:
#                 Network(int(batch), rate)
#             except KeyboardInterrupt:
#                 pass
#             rate /= 2
#         batch /= 2

#     print("---------------------------------------------------------------------")
# #     print(batch, rate)
#     try:
#         Network(16, 0.0003)
#     except KeyboardInterrupt:
#         pass

if __name__ == '__main__':
    main()

Instructions for updating:
Use tf.print instead of tf.Print. Note that tf.print returns a no-output operator that directly prints the output. Outside of defuns or eager mode, this operator will not be executed unless it is directly specified in session.run or used as a control dependency for other operators. This is only a concern in graph mode. Below is an example of how to ensure tf.print executes in graph mode:
```python
    sess = tf.Session()
    with sess.as_default():
        tensor = tf.range(10)
        print_op = tf.print(tensor)
        with tf.control_dependencies([print_op]):
          out = tf.add(tensor, tensor)
        sess.run(out)
    ```
Additionally, to use tf.print in python 2.7, users must make sure to import
the following:

  `from __future__ import print_function`

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).rep